# 

In [13]:

import os
import openai
openai.organization = "org-3vLJdrSekf44Opl39cqZZoqM"
openai.api_key = os.getenv("OPENAI_API_KEY")
model="gpt-4"
models = openai.Model.list()['data']



## Prints out available models

In [7]:
modelIds = str.join('\n',sorted(map(lambda m: m["id"], models)))
#print (modelIds)

## function example

In [15]:
chat_completion = openai.ChatCompletion.create(model=model, messages=[
    {"role": "system", "content": "You are an assistant who can only respond in limericks"},
    {"role": "user", "content": "what is capital of UK"}
])

# print the chat completion
print(chat_completion.choices[0].message.content)

The heart of the UK, it's fair,
Is a city of old charm and flair. 
Called London, you see, 
Full of history, it's key.
A visit will make you aware.


## Function calling

In [18]:
import openai
import json


# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    
    return json.dumps(weather_info)


def run_conversation():
    # Step 1: send the conversation and available functions to GPT
    messages = [{"role": "user", "content": "What's the weather like in Boston?"}]
    functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
    ]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call="auto",  # auto is default, but we'll be explicit
    )
    response_message = response["choices"][0]["message"]
    print(f" Response  from gpt is {response_message}")

    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        function_name = response_message["function_call"]["name"]
        fuction_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        function_response = fuction_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )
        print(f"The function response is {function_response}")

        # Step 4: send the info on the function call and function response to GPT
        messages.append(response_message)  # extend conversation with assistant's reply
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
        )  # get a new response from GPT where it can see the function response
        return second_response


print(run_conversation())

 Response  from gpt is {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n\"location\": \"Boston, MA\"\n}"
  }
}
The function response is {"location": "Boston, MA", "temperature": "72", "unit": null, "forecast": ["sunny", "windy"]}
{
  "id": "chatcmpl-7ugbgYwIsd4fH2djY43mqFNNknFv2",
  "object": "chat.completion",
  "created": 1693744504,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The weather in Boston is currently sunny and windy with a temperature of 72 degrees."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 71,
    "completion_tokens": 17,
    "total_tokens": 88
  }
}
